Pulling the data from drive

In [1]:
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import torchvision
import torchvision.datasets
import torchvision.transforms as transforms
from google.colab import drive
from PIL import Image, ImageOps
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
transform = transforms.Compose([transforms.PILToTensor()])

def manual_pull(iterable, indices):
    output=[]
    for i in indices:
        image=ImageOps.grayscale(iterable[i][0])
        image=transform(image)/255
        label=iterable[i][1]
        output.append((image,label))
    return output

classifier_dir="/content/drive/MyDrive/APS360 Group Project/Classified HMEs" #Temp model training data
classifier_data=torchvision.datasets.ImageFolder(classifier_dir)

In [3]:
print(classifier_data)

Dataset ImageFolder
    Number of datapoints: 920
    Root location: /content/drive/MyDrive/APS360 Group Project/Classified HMEs


In [4]:
np.random.seed(60) #Was 50 before
data_len=len(classifier_data)
indices=list(range(data_len))
np.random.shuffle(indices)
train_indices=indices[:840]
val_indices=indices[840:920]

train_c_set = manual_pull(classifier_data,train_indices)
val_c_set = manual_pull(classifier_data,val_indices)

img_to_tensor = transforms.ToTensor()

Baseline Model

In [5]:
class BaselineModel(nn.Module):
    def __init__(self):
        super(BaselineModel, self).__init__()
        self.layer1 = nn.Linear(520*173, 3000)
        self.layer2 = nn.Linear(3000, 200)
        self.layer3 = nn.Linear(200, 9)
    def forward(self, img):
        flattened = img.view(-1, 520*173)
        activation1 = F.relu(self.layer1(flattened))
        activation2 = F.relu(self.layer2(activation1))
        output = self.layer3(activation2)
        return output

model = BaselineModel()

In [6]:
def get_accuracy(data, model):
    correct = 0
    total = 0
    for imgs, labels in torch.utils.data.DataLoader(data):
        output = model(imgs) 
        #Puts the image through the model, one at a time, gets an output of a size 10 vector
        #select index with maximum prediction score
        pred = output.max(1, keepdim=True)[1] 
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += imgs.shape[0]
    return correct / total

In [11]:
def train_base(model, train_data, val_data, epochs=100, learning_rate=0.0005):
    criterion=nn.CrossEntropyLoss()
    optimizer=torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

    outputs=[]
    
    train_batch=torch.utils.data.DataLoader(train_data, 40)
    val_batch=torch.utils.data.DataLoader(val_data, 40)

    one_hot_encoding=torch.eye(9)

    for epoch in range(epochs):
        batch_no=0
        total_loss=0
        for imgs, label in train_batch:

            optimizer.zero_grad()

            label=one_hot_encoding[label]

            out=model(imgs)
            loss=criterion(out, label)
            #print(loss)
            loss.backward()
            optimizer.step()

            total_loss=total_loss+loss
            batch_no+=1

        train_acc=get_accuracy(train_data, model)
        val_acc=get_accuracy(val_data, model)

        print('Epoch:{}, Loss:{:.4f}, Train accuracy:{:.4f}, Validation accuracy:{:.4f}'.format(epoch+1, float(total_loss/batch_no), train_acc, val_acc))
        outputs.append((epoch, total_loss/batch_no, train_acc, val_acc))

    return outputs

In [12]:
train_base(model, train_data=train_c_set, val_data=val_c_set)

Epoch:1, Loss:2.1840, Train accuracy:0.1345, Validation accuracy:0.1625
Epoch:2, Loss:2.2113, Train accuracy:0.1345, Validation accuracy:0.1625
Epoch:3, Loss:2.1910, Train accuracy:0.1345, Validation accuracy:0.1625
Epoch:4, Loss:2.1906, Train accuracy:0.1345, Validation accuracy:0.1625
Epoch:5, Loss:2.1814, Train accuracy:0.1345, Validation accuracy:0.1625
Epoch:6, Loss:2.1762, Train accuracy:0.1345, Validation accuracy:0.1625
Epoch:7, Loss:2.1695, Train accuracy:0.1845, Validation accuracy:0.1625
Epoch:8, Loss:2.1594, Train accuracy:0.1607, Validation accuracy:0.1625
Epoch:9, Loss:2.1532, Train accuracy:0.1429, Validation accuracy:0.1625
Epoch:10, Loss:2.1450, Train accuracy:0.1857, Validation accuracy:0.1625
Epoch:11, Loss:2.1359, Train accuracy:0.2167, Validation accuracy:0.1875
Epoch:12, Loss:2.1252, Train accuracy:0.2226, Validation accuracy:0.2000
Epoch:13, Loss:2.1155, Train accuracy:0.2298, Validation accuracy:0.2125
Epoch:14, Loss:2.1033, Train accuracy:0.2286, Validation acc

[(0, tensor(2.1840, grad_fn=<DivBackward0>), 0.13452380952380952, 0.1625),
 (1, tensor(2.2113, grad_fn=<DivBackward0>), 0.13452380952380952, 0.1625),
 (2, tensor(2.1910, grad_fn=<DivBackward0>), 0.13452380952380952, 0.1625),
 (3, tensor(2.1906, grad_fn=<DivBackward0>), 0.13452380952380952, 0.1625),
 (4, tensor(2.1814, grad_fn=<DivBackward0>), 0.13452380952380952, 0.1625),
 (5, tensor(2.1762, grad_fn=<DivBackward0>), 0.13452380952380952, 0.1625),
 (6, tensor(2.1695, grad_fn=<DivBackward0>), 0.18452380952380953, 0.1625),
 (7, tensor(2.1594, grad_fn=<DivBackward0>), 0.16071428571428573, 0.1625),
 (8, tensor(2.1532, grad_fn=<DivBackward0>), 0.14285714285714285, 0.1625),
 (9, tensor(2.1450, grad_fn=<DivBackward0>), 0.18571428571428572, 0.1625),
 (10, tensor(2.1359, grad_fn=<DivBackward0>), 0.21666666666666667, 0.1875),
 (11, tensor(2.1252, grad_fn=<DivBackward0>), 0.2226190476190476, 0.2),
 (12, tensor(2.1155, grad_fn=<DivBackward0>), 0.22976190476190475, 0.2125),
 (13, tensor(2.1033, grad_

finish the training function, and write the get accuracy function